In [1]:
import model_function as mf

/home/ubuntu/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:43: DeprecationWarning: This module has been deprecated in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import pandas as pd
import psycopg2 as pg2

In [3]:
df = pd.read_csv('/data/npi_2013_clean1.csv')

/home/ubuntu/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
X_train, X_test = mf.make_train_test_dataset(df)

In [5]:
keep_cols = ['bene_count'
             , 'total_claim_count'
             , 'total_drug_cost'
             , 'total_day_supply'
             , 'brand_claim_count'
             , 'brand_claim_cost'
             , 'generic_claim_count'
             , 'generic_claim_cost'
             , 'other_claim_count'
             , 'other_claim_cost'
             , 'mapd_claim_count'
             , 'mapd_claim_cost'
             , 'pdp_claim_count'
             , 'pdp_claim_cost'
             , 'lis_claim_count'
             , 'lis_claim_cost'
             , 'nonlis_claim_count'
             , 'nonlis_claim_cost'
            ]

In [6]:
mod_if = mf.fit_model(X_train, keep_cols)

[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:   40.2s finished


In [7]:
conn = pg2.connect(dbname='medicare', user='postgres')
query = '''
        SELECT n.npi
            , indict.first_name
            , indict.last_name
            , indict.npi_status
        FROM indictments_2013 indict
            , npi_name n
        WHERE lower(indict.first_name) = lower(n.providerfirstname)
        AND lower(indict.last_name) = lower(n.providerlastnamelegalname)
        AND lower(indict.state) = lower(n.providerbusinessmailingaddressstatename);
        '''

In [8]:
df_labels = pd.read_sql_query(query, conn)

In [9]:
test_df = mf.prediction(X_test, keep_cols, mod_if)

model_function.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  test_df['prediction'] = model.predict(test_df[model_columns])


In [10]:
test_df = mf.check_labels(df_labels, test_df, keep_cols)

model_function.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  else 0 for row in test_df['npi']]


In [11]:
test_df[test_df['indicted']==True]

,Unnamed: 0,npi,last_name,first_name,entity_code,provider_zip,provider_city,specialty_desc,bene_count,total_claim_count,...,nonlis_claim_cost,brand_claim_cost_avg,generic_claim_cost_avg,other_claim_cost_avg,mapd_claim_cost_avg,pdp_claim_cost_avg,lis_claim_cost_avg,nonlis_claim_cost_avg,prediction,indicted
459746,333174,1053354787,GOLDFEIN,AARON,I,481242921,DEARBORN,General Practice,497,20657,...,77199.43,162.148990,19.399925,21.653271,39.874235,60.528605,60.136808,42.031533,0.713500,1
13269,40998,1881664829,ATAYA,HATEM,I,486619390,WEST BRANCH,Internal Medicine,868,17664,...,515616.34,192.936023,23.082508,29.143358,54.736709,67.127356,67.169662,59.395760,0.705946,1
526580,689114,1902838014,OGBUOKIRI,GODWIN,I,701263744,NEW ORLEANS,General Surgery,107,1355,...,2330.62,214.344588,22.366400,33.475198,50.298379,62.034269,65.986955,51.478635,0.414670,1
980640,755913,1427136043,QADIR,RIZWAN,I,483141723,STERLING HEIGHTS,Neurology,426,3863,...,260697.01,945.720550,42.924785,23.248386,244.113129,255.598196,273.906044,233.294313,0.534572,1
936622,66443,1497781447,BAZZI,MOHAMAD,I,481873338,CANTON,Plastic and Reconstructive Surgery,64,161,...,34220.89,200.472847,21.124370,24.456168,32.103744,42.045297,52.568557,33.793751,0.380008,1
313717,85833,1427000819,BINDER,WILLIAM,I,481985753,YPSILANTI,Family Practice,73,926,...,4899.50,181.149499,22.071589,28.569992,50.773910,58.693464,59.878618,52.614634,0.361441,1
1030910,206541,1548427164,DANESHVAR,GERALD,I,482123221,HAMTRAMCK,Geriatric Medicine,731,12133,...,104944.78,156.001166,23.273950,28.040818,50.013939,56.604035,54.972580,54.413811,0.631432,1
338511,329079,1528044815,GLAZER,ROBERT,I,905055023,TORRANCE,Family Practice,474,4723,...,282168.83,181.149499,22.071589,28.569992,50.773910,58.693464,59.878618,52.614634,0.547321,1
